### Imports

In [42]:
import psycopg2
import pandas as pd
import numpy as np
import warnings

from config import config
from datetime import datetime, timedelta

warnings.filterwarnings("ignore")


### Define connect method to connect to the postgres database and return the connection

In [9]:
def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)

        return conn

        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

### Connect to the database

In [16]:
conn = connect()

Connecting to the PostgreSQL database...


### Create the database tables

In [41]:
cur = conn.cursor()

# Country Table
cur.execute("""CREATE TABLE Country (
                Country_key int PRIMARY KEY,
                Name varchar(255),
                Region varchar(255),
                Continent varchar(255),
                Currency varchar(255),
                Capital varchar(255),
                Total_population int,
                Birth_rate float,
                Gross_national_income float,
                Life_expectancy_at_birth float,
                Labor_force_total int,
                Human_capital_index int,
                Population_grown_annual float
            );""")

# Month Table
cur.execute("""CREATE TABLE Month (
                    Month_Key int PRIMARY KEY,
                    Name varchar(255),
                    Quarter int,
                    Year int,
                    Decade int
                );""")

# Education Table
cur.execute("""CREATE TABLE Education (
                    Education_Key int PRIMARY KEY,
                    Total_Literacy_Rate float,
                    Male_Literacy_Rate float,
                    Female_Literacy_Rate float,
                    Primary_School_Enrollment float,
                    Post_Secondary_School_Enrollment float,
                    Public_Education_Spending float,
                    Pop_compuslory_school_age_total int,
                    Pop_offical_entrance_age_primary_total int,
                    Pop_offical_entrance_age_secondary_total int,
                    Teachers_primary_total int,
                    Teachers_secondary_total int
                );""")

# Health Table
cur.execute("""CREATE TABLE Health (
                    Health_Key int PRIMARY KEY,
                    Domestic_Health_Expenditure float,
                    Hospital_Beds float,
                    Immunization_attr_Hep int,
                    Immunization_attr_DPT int,
                    Immunization_attr_Measles int,
                    Immunization_attr_Polio int,
                    Num_Surgical_procedures int,
                    Num_Death_infant int,
                    Num_Death_stilbirths int,
                    Num_Death_Elderly int,
                    Num_health_professionals_Nurses float,
                    Num_health_professionals_Physicians float,
                    Prevalence_health_condition_overweight float,
                    Prevalence_health_condition_diabetes float,
                    Prevalence_health_condition_hiv float,
                    Adults_HIV_15up float,
                    Adults_new_HIV_15up float,
                    Children_HIV_under15 float,
                    Children_new_HIV_under15 float,
                    Homelessness_rate_male float,
                    Homelessness_rate_female float,
                    Homelessness_rate_total float,
                    CrimeRate float,
                    Cost_of_living_index float
                );""")

# Quality_of_Life Table
cur.execute("""CREATE TABLE Quality_of_Life(
                    Quality_of_Life_Key int PRIMARY KEY,
                    Access_to_Drinking_Water float,
                    Access_to_Sanitation float,
                    Access_to_Basic_Handwashing_Facilities float,
                    Unemployment_Rate_F float,
                    Unemployment_Rate_M float,
                    Unemployment_Rate_T float,
                    Access_to_Electricity_Total float,
                    Access_to_Electricity_Urban float,
                    Access_to_Electricity_Rural float,
                    Part_Time_Employment_T float, 
                    Part_Time_Employment_F float,    
                    Part_Time_Employment_M float
                );""")

# Population Table
cur.execute("""CREATE TABLE Population(
                    Population_Key int PRIMARY KEY,
                    Life_Expectancy_At_Birth_F float,
                    Life_Expectancy_At_Birth_M float,
                    Life_Expectancy_At_Birth_T float,
                    Net_Migration int,
                    Population_ages_0_15 int,
                    Population_ages_16_30 int,
                    Population_ages_31_64 int,
                    Population_ages_65_up int,
                    Rural_Population float,
                    Rural_Population_Growth_Rate float,
                    Rural_Poverty_Rate float,
                    Urban_Population float,
                    Urban_Population_Growth_Rate float,
                    Urban_Poverty_Rate float
                );""")

# Event Table
cur.execute("""CREATE TABLE Event (
                    Event_key int PRIMARY KEY,
                    Name varchar(255),
                    Disaster_Type varchar(255),
                    Start_Day int,
                    End_Day int,
                    Start_Month int,
                    End_Month int,
                    Start_Year int,
                    End_Year int,
                    Disaster_Subgroup varchar(255),
                    Total_Deaths int,
                    No_Injured int,
                    No_Affected int
                );""")

# Fact Table
cur.execute("""CREATE TABLE Fact_Table (
                    Month_Key int,
                    Country_Key int,
                    Education_Key int,
                    Population_Key int,
                    Quality_of_Life_Key int,
                    Health_Key int,
                    Event_Key int,
                    Quality_of_life int,
                    Development_Index int,
                    Human_Development_Index int,
                    CONSTRAINT fk_month FOREIGN KEY(Month_Key) REFERENCES Month(Month_Key),
                    CONSTRAINT fk_country FOREIGN KEY(Country_Key) REFERENCES Country(Country_Key),
                    CONSTRAINT fk_education FOREIGN KEY(Education_Key) REFERENCES Education(Education_Key),
                    CONSTRAINT fk_population FOREIGN KEY(Population_Key) REFERENCES Population(Population_Key),
                    CONSTRAINT fk_quality_of_life FOREIGN KEY(Quality_of_Life_Key) REFERENCES Quality_Of_Life(Quality_of_Life_Key),
                    CONSTRAINT fk_health FOREIGN KEY(Health_Key) REFERENCES Health(Health_Key),
                    CONSTRAINT fk_event FOREIGN KEY(Event_Key) REFERENCES Event(Event_Key)
                );""")

cur.close()


In [40]:
cur = conn.cursor()

# cur.execute("DROP TABLE Fact_Table")
# cur.execute("DROP TABLE Country")
# cur.execute("DROP TABLE Month")
# cur.execute("DROP TABLE Education")
# cur.execute("DROP TABLE Health")
# cur.execute("DROP TABLE Quality_of_Life")
# cur.execute("DROP TABLE Population")
# cur.execute("DROP TABLE Event")

cur.close()


Rollback incase that shit breaks

In [23]:
cur.execute("ROLLBACK")

### Read the data from our CSVs using pandas

In [206]:
country_info_data = pd.read_csv("Data/CountryInfo.csv")
development_index_data = pd.read_csv("Data/Development Index.csv")
emdat_data = pd.read_csv("Data/emdat_public_2022_03_14_query_uid-tJR2bL.csv")
hdi_data = pd.read_csv("Data/human-development-index-escosura.csv")
qol_index_data = pd.read_csv("Data/Quality Of Life Index.csv")
wb_education_data = pd.read_csv("Data/WorldBankEducationStatisticsAllIndicators_Data.csv")
wb_hnps_data = pd.read_csv("Data/WorldBankHealthNutritionAndPopulationStatistics_Data.csv")
wb_poverty_and_equity = pd.read_csv("Data/WorldBankPovertyAndEquity_Data.csv")
wb_world_development_indicators = pd.read_csv("Data/WorldBankWorldDevelopmentIndicators_Data.csv")

### Define our list of countries

In [222]:
countries = [
    "Canada",
    "United States",
    "Mexico",
    "Indonesia",
    "Angola",
    "Cambodia",
    "Thailand",
    "South Africa",
    "Zimbabwe"
]

### A function to convert the data frame structure

In [234]:
def convert_dataframe_structure(df):
    dataframes = []


    for country in countries:
        temp_df = df.loc[(df['Country Name'] == country)]

        temp_df = temp_df.T
        temp_df = temp_df.drop("Country Name")


        new_header = temp_df.iloc[0]
        temp_df = temp_df[1:]
        temp_df.columns = new_header

        temp_df["Country"] = country

        dataframes.append(temp_df)

    return pd.concat(dataframes)

### Create Dimension DataFrames

In [177]:
country_dimension_df = pd.DataFrame(
    columns=
    [
        "CountryKey", 
        "Name", 
        "Region", 
        "Continent", 
        "Currency", 
        "Capital", 
        "Total_Population", 
        "Birth_Rate", 
        "Gross_National_Income", 
        "Life_Expectancy_at_Birth",
        "Labor_Force",
        "Human_Capital_Index",
        "Population_Growth"
    ]
)

month_dimension_df = pd.DataFrame(
    columns=
    [
        "MonthKey",
        "Name",
        "Quarter",
        "Year",
        "Decade"
    ]
)

education_dimension_df = pd.DataFrame(
    columns=
    [
        "Education_Key",
        "Total_Literacy_Rate",
        "Male_Literacy_Rate",
        "Female_Literacy_Rate",
        "Primary_School_Enrollment",
        "Secondary_School_Enrollment",
        "Post-Secondary_School_Enrollment",
        "Public_Education_Spending",
        "Compulsory_School_Age_Population",
        "Official_Entrance_Age_Primary_Education_Population",
        "Official_Entrance_Age_Secondary_Education_Population",
        "Teachers_in_Primary_Education",
        "Teachers_in_Secondary_Education"
    ]
)

health_dimension_df = pd.DataFrame(
    columns=
    [ 
        "Health_Key"
        "Domestic_Health_Expenditure",
        "Hospital_Beds",
        "Immunization_Hep",
        "Immunization_DPT",
        "Immunization_Measles",
        "Immunization_Polio",
        "No_Surgical_Procedures",
        "No_Deaths_Infant",
        "No_Deaths_Stillbirth",
        "No_Deaths_20-24",
        "No_Nurses",
        "No_Physicians",
        "Prevalence_Overweight",
        "Prevalence_Diabetes",
        "Prevalence_HIV",
        "Adults_Living_With_HIV",
        "Adults_Newly_Infected_With_HIV",
        "Children_Living_With_HIV",
        "Children_Newly_Infected_With_HIV"
    ]
)

quality_of_life_df = pd.DataFrame(
    columns=
    [
        "Quality_Of_Life_Key"
        "Access_To_Drinking_Water",
        "Access_To_Sanitation",
        "Access_To_Basic_Handwashing_Facilities",
        "Unemployment_Rate_Female",
        "Unemployment_Rate_Male",
        "Unemployment_Rate_Total",
        "Maternal_Leave_Benefits",
        "Access_To_Electricity_Total",
        "Access_To_Electricity_Urban",
        "Access_To_Electricity_Rural",
        "Part_Time_Employment_Total",
        "Part_Time_Employment_Female",
        "Part_Time_Employment_Male"
    ]
)

population_df = pd.DataFrame(
    columns = 
    [
        "Country", #
        "Population_Key",
        "Life_Expectancy_attributes_F",
        "Life_Expectancy_attributes_M",
        "Life_Expectancy_attributes_T",
        "Net_Migration",
        "Population_Statistics_0_15",
        "Population_Statistics_16_30",
        "Population_Statistics_31_64",
        "Population_Statistics_65",
        "Rural_Population",
        "Rural_Population_Growth_Rate",
        "Rural_Poverty_Rate",
        "Urban_Population",
        "Urban_Population_Growth_Rate",
        "Urban_Poverty_Rate"
    ],
    index=
    [
        "2005 [YR2005]",
        "2006 [YR2006]",
        "2007 [YR2007]",
        "2008 [YR2008]",
        "2009 [YR2009]",
        "2010 [YR2010]",
        "2011 [YR2011]",
        "2012 [YR2012]",
        "2013 [YR2013]",
        "2014 [YR2014]",
        "2015 [YR2015]",
        "2016 [YR2016]",
        "2017 [YR2017]",
        "2018 [YR2018]",
        "2019 [YR2019]",
        "2020 [YR2020]"
    ]
)

event_df = pd.DataFrame(
    columns=
    [
        "EventKey",
        "Name", 
        "Disaster_Type", 
        "Start_date", 
        "End_date", 
        "Start_Month", 
        "End_month", 
        "Disaster_Subgroup",
        "Total_Deaths",
        "No_Injured",
        "No_Affected"
    ]
)


### Populate the Population DataFrame

In [239]:
wb_hnps_population_data = wb_hnps_data.loc[
    (wb_hnps_data['Series Name'] == "Life expectancy at birth, female (years)") | 
    (wb_hnps_data['Series Name'] == "Life expectancy at birth, male (years)") | 
    (wb_hnps_data['Series Name'] == "Life expectancy at birth, total (years)") | 
    (wb_hnps_data['Series Name'] == "Net migration") | 
    (wb_hnps_data['Series Name'] == "Population ages 00-14, total") | 
    (wb_hnps_data['Series Name'] == "Population ages 15-19, female") | 
    (wb_hnps_data['Series Name'] == "Population ages 15-19, male") | 
    (wb_hnps_data['Series Name'] == "Population ages 20-24, female") | 
    (wb_hnps_data['Series Name'] == "Population ages 20-24, male") | 
    (wb_hnps_data['Series Name'] == "Population ages 25-29, female") | 
    (wb_hnps_data['Series Name'] == "Population ages 25-29, male") | 
    (wb_hnps_data['Series Name'] == "Population ages 30-34, female") | 
    (wb_hnps_data['Series Name'] == "Population ages 30-34, male") | 
    (wb_hnps_data['Series Name'] == "Population ages 35-39, female") | 
    (wb_hnps_data['Series Name'] == "Population ages 35-39, male") | 
    (wb_hnps_data['Series Name'] == "Population ages 40-44, female") | 
    (wb_hnps_data['Series Name'] == "Population ages 40-44, male") | 
    (wb_hnps_data['Series Name'] == "Population ages 45-49, female") | 
    (wb_hnps_data['Series Name'] == "Population ages 45-49, male") | 
    (wb_hnps_data['Series Name'] == "Population ages 50-54, female") | 
    (wb_hnps_data['Series Name'] == "Population ages 50-54, male") | 
    (wb_hnps_data['Series Name'] == "Population ages 55-59, female") | 
    (wb_hnps_data['Series Name'] == "Population ages 55-59, male") | 
    (wb_hnps_data['Series Name'] == "Population ages 60-64, female") | 
    (wb_hnps_data['Series Name'] == "Population ages 60-64, male") | 
    (wb_hnps_data['Series Name'] == "Population ages 65 and above, female") | 
    (wb_hnps_data['Series Name'] == "Population ages 65 and above, male") | 
    (wb_hnps_data['Series Name'] == "Rural population (% of total population)") | 
    (wb_hnps_data['Series Name'] == "Rural population growth (annual %)") | 
    (wb_hnps_data['Series Name'] == "Rural poverty headcount ratio at national poverty lines (% of rural population)") |
    (wb_hnps_data['Series Name'] == "Urban population (% of total population)") | 
    (wb_hnps_data['Series Name'] == "Urban population growth (annual %)") |  
    (wb_hnps_data['Series Name'] == "Urban poverty headcount ratio at national poverty lines (% of urban population)")
].drop(columns=['Series Code', 'Country Code'])

population_df = convert_dataframe_structure(wb_hnps_population_data)

population_df.head()


Series Name,"Life expectancy at birth, total (years)","Life expectancy at birth, female (years)","Life expectancy at birth, male (years)",Net migration,"Population ages 00-14, total","Population ages 15-19, female","Population ages 15-19, male","Population ages 20-24, female","Population ages 20-24, male","Population ages 25-29, female",...,"Population ages 60-64, male","Population ages 65 and above, female","Population ages 65 and above, male",Rural population (% of total population),Rural population growth (annual %),Rural poverty headcount ratio at national poverty lines (% of rural population),Urban poverty headcount ratio at national poverty lines (% of urban population),Urban population (% of total population),Urban population growth (annual %),Country
2005 [YR2005],80.1926829268293,82.6,77.9,..,5699388,1057095,1119679,1090497,1133619,1067433,...,753553,2388516,1841079,19.878,0.557844254173616,..,..,80.122,1.04061858829188,Canada
2006 [YR2006],80.3439024390244,82.7,78.1,..,5667703,1071021,1133820,1106962,1152539,1088653,...,791162,2431677,1890093,19.787,0.551491248878675,..,..,80.213,1.12384675481251,Canada
2007 [YR2007],80.5439024390244,82.9,78.3,1326431,5635757,1083177,1144449,1118954,1169178,1112280,...,832820,2477606,1941565,19.604,0.0419781487099549,..,..,80.396,1.19901884209158,Canada
2008 [YR2008],80.6951219512195,83,78.5,..,5616013,1091597,1150113,1129840,1185831,1137922,...,876935,2532188,2000002,19.422,0.150191983656897,..,..,80.578,1.30902990520455,Canada
2009 [YR2009],80.9951219512195,83.3,78.8,..,5608093,1092176,1147819,1140955,1202142,1162681,...,919198,2597144,2066447,19.242,0.210657043137183,..,..,80.758,1.36489394771959,Canada


### Populate the Health DataFrame

In [241]:
wb_hnps_health_data = wb_hnps_data.loc[
    (wb_hnps_data['Series Name'] == "Domestic general government health expenditure (% of GDP)") | 
    (wb_hnps_data['Series Name'] == "Hospital beds (per 1,000 people)") | 
    (wb_hnps_data['Series Name'] == "Immunization, HepB3 (% of one-year-old children)") | 
    (wb_hnps_data['Series Name'] == "Immunization, DPT (% of children ages 12-23 months)") | 
    (wb_hnps_data['Series Name'] == "Immunization, measles (% of children ages 12-23 months)") | 
    (wb_hnps_data['Series Name'] == "Immunization, Pol3 (% of one-year-old children)") | 
    (wb_hnps_data['Series Name'] == "Number of surgical procedures (per 100,000 population)") | 
    (wb_hnps_data['Series Name'] == "Number of infant deaths") | 
    (wb_hnps_data['Series Name'] == "Number of stillbirths") | 
    (wb_hnps_data['Series Name'] == "Number of deaths ages 20-24 years") | 
    (wb_hnps_data['Series Name'] == "Nurses and midwives (per 1,000 people)") | 
    (wb_hnps_data['Series Name'] == "Physicians (per 1,000 people)") | 
    (wb_hnps_data['Series Name'] == "Prevalence of overweight (% of adults)") | 
    (wb_hnps_data['Series Name'] == "Diabetes prevalence (% of population ages 20 to 79)") | 
    (wb_hnps_data['Series Name'] == "Prevalence of HIV, total (% of population ages 15-49)") | 
    (wb_hnps_data['Series Name'] == "Adults (ages 15+) living with HIV") | 
    (wb_hnps_data['Series Name'] == "Adults (ages 15-49) newly infected with HIV") | 
    (wb_hnps_data['Series Name'] == "Children (0-14) living with HIV") | 
    (wb_hnps_data['Series Name'] == "Children (ages 0-14) newly infected with HIV")
].drop(columns=['Series Code', 'Country Code'])

health_df = convert_dataframe_structure(wb_hnps_health_data)

health_df.head()

Series Name,Domestic general government health expenditure (% of GDP),"Hospital beds (per 1,000 people)","Immunization, Pol3 (% of one-year-old children)","Immunization, measles (% of children ages 12-23 months)","Immunization, HepB3 (% of one-year-old children)","Immunization, DPT (% of children ages 12-23 months)","Number of surgical procedures (per 100,000 population)",Number of infant deaths,Number of stillbirths,Number of deaths ages 20-24 years,"Physicians (per 1,000 people)","Nurses and midwives (per 1,000 people)",Prevalence of overweight (% of adults),Diabetes prevalence (% of population ages 20 to 79),"Prevalence of HIV, total (% of population ages 15-49)",Adults (ages 15+) living with HIV,Adults (ages 15-49) newly infected with HIV,Children (0-14) living with HIV,Children (ages 0-14) newly infected with HIV,Country
2005 [YR2005],6.59905815,3.1,93,94,14,93,..,1800,1037,1307,..,9.9467,58.6,..,..,..,..,..,..,Canada
2006 [YR2006],6.4866991,3.02,95,93,14,95,..,1825,1083,1311,1.9086,10.0236,59.2,..,..,..,..,..,..,Canada
2007 [YR2007],6.57271051,2.96,99,94,14,94,..,1845,1129,1307,..,10.1208,59.7,..,..,..,..,..,..,Canada
2008 [YR2008],6.70117044,2.85,96,93,28,92,..,1860,1138,1298,1.959,10.289,60.2,..,..,..,..,..,..,Canada
2009 [YR2009],7.52118206,2.8,93,91,42,91,..,1869,1126,1288,..,10.3743,60.7,..,..,..,..,..,..,Canada


### Populate the Event DataFrame

In [61]:
emdat_data.columns

Index(['Dis No', 'Year', 'Seq', 'Glide', 'Disaster Group', 'Disaster Subgroup',
       'Disaster Type', 'Disaster Subtype', 'Disaster Subsubtype',
       'Event Name', 'Country', 'ISO', 'Region', 'Continent', 'Location',
       'Origin', 'Associated Dis', 'Associated Dis2', 'OFDA Response',
       'Appeal', 'Declaration', 'Aid Contribution', 'Dis Mag Value',
       'Dis Mag Scale', 'Latitude', 'Longitude', 'Local Time', 'River Basin',
       'Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month',
       'End Day', 'Total Deaths', 'No Injured', 'No Affected', 'No Homeless',
       'Total Affected', 'Reconstruction Costs ('000 US$)',
       'Reconstruction Costs, Adjusted ('000 US$)',
       'Insured Damages ('000 US$)', 'Insured Damages, Adjusted ('000 US$)',
       'Total Damages ('000 US$)', 'Total Damages, Adjusted ('000 US$)', 'CPI',
       'Adm Level', 'Admin1 Code', 'Admin2 Code', 'Geo Locations'],
      dtype='object')

In [83]:
event_df = emdat_data[[
    "Disaster Subgroup",
    "Event Name",
    "Country",
    "Disaster Type", 
    "Start Year", 
    "Start Month", 
    "Start Day", 
    "End Year", 
    "End Month", 
    "End Day", 
    "Total Deaths",
    "No Injured",
    "No Affected"]]


event_df.head(15)

,Disaster Subgroup,Event Name,Country,Disaster Type,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,No Injured,No Affected
0,Hydrological,NaN,Angola,Flood,2005,1.0,27.0,2005,3.0,28.0,NaN,NaN,NaN
1,Technological,NaN,Angola,Transport accident,2005,2.0,8.0,2005,2.0,8.0,20.0,70.0,NaN
2,Geophysical,NaN,Indonesia,Earthquake,2005,3.0,28.0,2005,3.0,28.0,915.0,1146.0,104167.0
3,Geophysical,NaN,Indonesia,Earthquake,2005,1.0,23.0,2005,1.0,23.0,1.0,4.0,680.0
4,Hydrological,NaN,Indonesia,Landslide,2005,2.0,21.0,2005,2.0,21.0,143.0,NaN,NaN
5,Technological,Casa 212,Indonesia,Transport accident,2005,3.0,22.0,2005,3.0,22.0,15.0,3.0,NaN
6,Technological,NaN,Indonesia,Transport accident,2005,3.0,13.0,2005,3.0,13.0,33.0,NaN,NaN
7,Hydrological,NaN,United States of America (the),Flood,2005,2.0,17.0,2005,2.0,23.0,9.0,NaN,150.0
8,Hydrological,NaN,United States of America (the),Flood,2005,1.0,7.0,2005,1.0,11.0,28.0,8.0,500.0
9,Meteorological,NaN,United States of America (the),Storm,2005,1.0,22.0,2005,1.0,24.0,20.0,NaN,NaN
